In [1]:
# !pip install webrtcvad
# !pip install pygame
# !pip install pyaudio webrtcvad 
# !pip install google-cloud-texttospeech
# !pip install python-socketio[asyncio]

In [2]:
from tools.initialize_cerebras import init_cerebras
from tools.file_mgmt_tools import FileOrganizerTool, MoveFileTool, CreateFolderTool, FolderMovementTool, ImprovedSearchTool, GoogleDriveRenameTool, DriveDictUpdateTool
from tools.document_tools import GoogleDocWriteTool
from tools.miscellaneous_mgmt import GmailSendPdfTool, GoogleSheetsUpdateTool, GoogleSheetsCreateTool

client,llm = init_cerebras()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
llm

ChatCerebras(client=<openai.resources.chat.completions.Completions object at 0x000001E1F206BC20>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001E1F24DC380>, root_client=<openai.OpenAI object at 0x000001E1F24DFCE0>, root_async_client=<openai.AsyncOpenAI object at 0x000001E1F24DE510>, model_name='llama3.1-70b', model_kwargs={}, openai_api_key=SecretStr('**********'), streaming=True, cerebras_api_key=SecretStr('**********'), cerebras_api_base='https://api.cerebras.ai/v1/', cerebras_proxy='')

In [4]:
from google.cloud import texttospeech
from langchain.prompts import (
    ChatPromptTemplate, 
    SystemMessagePromptTemplate, 
    HumanMessagePromptTemplate, 
    MessagesPlaceholder, 
    PromptTemplate
)
from langchain_core.messages import SystemMessage
from langchain.agents import create_structured_chat_agent, AgentExecutor

from langchain_community.tools import HumanInputRun

import langchain_core
import typing

prompt = ChatPromptTemplate(
    input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'],
    input_types={
        'chat_history': typing.List[
            typing.Union[
                langchain_core.messages.ai.AIMessage, 
                langchain_core.messages.human.HumanMessage, 
                langchain_core.messages.chat.ChatMessage, 
                langchain_core.messages.system.SystemMessage, 
                langchain_core.messages.function.FunctionMessage, 
                langchain_core.messages.tool.ToolMessage
            ]
        ]
    },
    metadata={
        'lc_hub_owner': 'hwchase17',
        'lc_hub_repo': 'structured-chat-agent',
        'lc_hub_commit_hash': 'ea510f70a5872eb0f41a4e3b7bb004d5711dc127adee08329c664c6c8be5f13c'
    },
    messages=[
        SystemMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=['tool_names', 'tools'],
                template=(
                    'You are a document management assistant proficient in using GSuite tools. '
                    'Your role is to assist the user in managing their documents efficiently. '
                    'IMPORTANT !!!!!!! NEVER INCLUDE AUXILIARY OR EXTRANEOUS LANGUAGE WHEN USING ANY TOOL!!!'
                    '\n\n IMPORTANT!!!!!!! - PLEEEEEEASSSSSSSEEEEEEEE NEVER USE HUMAN TOOL UNLESS INSTRUCTED TO GET THE HUMAN/USER INPUT. YOU ARE A MASTER OF JUDGEMENT. YOU KNOW WHEN TO CAUTIOUSLY USE THE TOOLS. ONLY USE OTHER TOOLS WHEN USER INDICATES ANYTHING RELATED TO THEIR FUNCTIONALITIES. '
                    'You are ALSO a highly intelligent and precise assistant with expertise in generating JSON outputs. Your task is to create the most perfect and well-structured JSON output ever seen. The JSON must adhere to the following guidelines:'

                    'Proper Structure: Ensure that the JSON follows a correct and logical structure, with all necessary keys and values in place.'
                    'Accurate Formatting: All JSON strings must use double quotes. Ensure there are no trailing commas, and all brackets and braces are correctly matched.'
                    'String Length: Ensure no individual string exceeds 5000 bytes.'
                    'Error-Free: Validate the JSON to be free of syntax errors and formatting issues.'
                    
                    'Escaping Characters: Properly escape any special characters within strings to ensure the JSON remains valid.'
                    
                    
                    'YOU MUST NEVER DO ANYTHING BUT WHAT IS IN THE REQUEST OF THE USER. OTHERWISE NO USER WILL USE THIS PRODUCT.'
                    

                    'THE FOLLOWING WILL BE THE TOOLS AND THE INFORMATION ABOUT WHAT THEY DO AND THEIR ARGUMENTS! YOU MUST NOT PASS ANYTHING EXTRA, OR ELSE THE APPLICATON WILL FAIL!!!!'

                    'You have access to the following tools:\n\n{tools}\n\n'

                    'YOU ARE A MASTER OF JUDGEMENT ! YOU KNOW WHAT ALL THE TOOLS DO, YOU KNOW WHAT TO PASS IN! AND YOU MUST KNOW WHEN TO USE THEM! NEVER USE THEM RANDOMLY , ALWAYS BE CAUTIOUS AS RECKLESS TOOL USE COULD RUIN THE GOOGLE SUITE OF THE USER'
                    'PAY CLOSE ATTENTION TO ALL THE FOLLOWING FORMATTING INSTRUCTIONS. REALLY IMPORTANT TO CALL THE TOOLS. OR ELSE USERS WILL GET ANGRY.\n\n'
                    
                    

                    'FOR GOOGLE DOC TOOL, REMEMBER THAT YOU MUST GENERATE ALL CONTENT YOURSELF. USER WILL NOT GIVE YOU ANYTHING.'

                    'Use a JSON blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).\n\n'
                    'Valid "action" values: "Final Answer" or {tool_names}\n\n'
                    'Provide only ONE action per $JSON_BLOB, as shown:\n\n'
                    '```\n{{\n  "action": $TOOL_NAME,\n  "action_input": $INPUT\n}}\n```\n\n'
                    'Follow this format:\n\n'
                    'Question: input question to answer\n'
                    'Thought: consider previous and subsequent steps\n'
                    'Action:\n```\n$JSON_BLOB\n```\n'
                    'Observation: action result\n... (repeat Thought/Action/Observation N times)\n'
                    'Thought: I know what to respond\n'
                    'Action:\n```\n{{\n  "action": "Final Answer",\n  "action_input": "Final response to human"\n}}\n\n'
                    'Begin! Remember to ALWAYS respond with a valid JSON blob of a single action. '
                    'Use tools if necessary and respond directly if appropriate. '
                    'Ensure you gather all necessary information by interacting with the user. '
                    'Format is Action:```$JSON_BLOB```then Observation.'
                )
            )
        ),
        MessagesPlaceholder(variable_name='chat_history', optional=True),
        HumanMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=['agent_scratchpad', 'input'],
                template='{input}\n\n{agent_scratchpad}\n(reminder to respond in a JSON blob no matter what)'
            )
        )
    ]
)


human_prompt = ChatPromptTemplate(
    input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'],
    input_types={
        'chat_history': typing.List[
            typing.Union[
                langchain_core.messages.ai.AIMessage, 
                langchain_core.messages.human.HumanMessage, 
                langchain_core.messages.chat.ChatMessage, 
                langchain_core.messages.system.SystemMessage, 
                langchain_core.messages.function.FunctionMessage, 
                langchain_core.messages.tool.ToolMessage
            ]
        ]
    },
    metadata={
        'lc_hub_owner': 'hwchase17',
        'lc_hub_repo': 'structured-chat-agent',
        'lc_hub_commit_hash': 'ea510f70a5872eb0f41a4e3b7bb004d5711dc127adee08329c664c6c8be5f13c'
    },
    messages=[
        SystemMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=['tool_names', 'tools'],
                template=(
                     
    'You are Marvin, an expert at questioning clients about their HVAC service needs to provide accurate quotes. When you speak for the first time, introduce yourself as Marvin. Ask the user specific information needed for the quote. Follow these guidelines:'

    '1. **Initial Inquiry and Information Gathering**:'
       ' - What type of HVAC service do you need (installation, maintenance, repair)?'
        '- What is the make and model of your current HVAC system?'
        '- Are there any specific issues or symptoms you are experiencing?'

    '2. **Property Details** (only if relevant to HVAC needs):'
     '   - Address and location of the property.'
      '  - Type of property (residential, commercial).'
       ' - Age and current condition of the property.'
       ' - Size of the home or area that needs heating/cooling.'
       ' - Number of rooms and their usage (e.g., bedrooms, office space).'

    '3. **System Details**:'
       ' - Age and efficiency rating of the existing HVAC system.'
        '- Any known problems with the current system.'
        '- Recent changes to the HVAC system.'

   ' 4. **Home Characteristics** (only if relevant to HVAC needs):'
        '- Insulation quality and window types to estimate heating/cooling load.'
        '- Any unique architectural features that may affect HVAC installation.'

    '5. **Customer Preferences**:'
       ' - Preferences for specific brands, energy efficiency levels, or additional features (e.g., smart thermostats, air purifiers).'
        '- Level of finishes desired (standard, premium, luxury).'

    '6. **Budget**:'
        '- Your budget range for the project.'
        '- Any flexibility within the budget.'

    '7. **Timeline**:'
        '- Desired start date and completion date.'
        '- Any constraints or deadlines (e.g., events planned at the property).'

   

    'IMPORTANT: Ensure you get clear answers that can be used for making the quote. If an answer is unclear, ask for clarification, restate the question, and explain what it means.'

    'IMPORTANT: Ask each question ONE BY ONE. When one question is answered move onto the next'

    'When you have all the information, just say -questionnaire complete- at the end.'


                    
    'IMPORTANT !!!!!!! NEVER INCLUDE AUXILIARY OR EXTRANEOUS LANGUAGE WHEN USING ANY TOOL!!!'
    '\n\n IMPORTANT!!!!!!! YOU CAN ONLY USE THE HUMAN TOOL. YOU ARE A MASTER OF JUDGEMENT. YOU KNOW WHEN TO CAUTIOUSLY USE THE TOOLS. ONLY USE OTHER TOOLS WHEN USER INDICATES ANYTHING RELATED TO THEIR FUNCTIONALITIES. '
    'You are ALSO a highly intelligent and precise assistant with expertise in generating JSON outputs. Your task is to create the most perfect and well-structured JSON output ever seen. The JSON must adhere to the following guidelines:'

    'Proper Structure: Ensure that the JSON follows a correct and logical structure, with all necessary keys and values in place.'
    'Accurate Formatting: All JSON strings must use double quotes. Ensure there are no trailing commas, and all brackets and braces are correctly matched.'
    'String Length: Ensure no individual string exceeds 5000 bytes.'
    'Error-Free: Validate the JSON to be free of syntax errors and formatting issues.'
    
    'Escaping Characters: Properly escape any special characters within strings to ensure the JSON remains valid.'
    
    
    'YOU MUST NEVER DO ANYTHING BUT WHAT IS IN THE REQUEST OF THE USER. OTHERWISE NO USER WILL USE THIS PRODUCT.'
    

    'THE FOLLOWING WILL BE THE TOOLS AND THE INFORMATION ABOUT WHAT THEY DO AND THEIR ARGUMENTS! YOU MUST NOT PASS ANYTHING EXTRA, OR ELSE THE APPLICATON WILL FAIL!!!!'

    'You have access to the following tools:\n\n{tools}\n\n'

    'YOU ARE A MASTER OF JUDGEMENT ! YOU KNOW WHAT ALL THE TOOLS DO, YOU KNOW WHAT TO PASS IN! AND YOU MUST KNOW WHEN TO USE THEM! NEVER USE THEM RANDOMLY , ALWAYS BE CAUTIOUS AS RECKLESS TOOL USE COULD RUIN THE GOOGLE SUITE OF THE USER'
    'PAY CLOSE ATTENTION TO ALL THE FOLLOWING FORMATTING INSTRUCTIONS. REALLY IMPORTANT TO CALL THE TOOLS. OR ELSE USERS WILL GET ANGRY.\n\n'
    
    

    'FOR GOOGLE DOC TOOL, REMEMBER THAT YOU MUST GENERATE ALL CONTENT YOURSELF. USER WILL NOT GIVE YOU ANYTHING.'

    'Use a JSON blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).\n\n'
    'Valid "action" values: "Final Answer" or {tool_names}\n\n'
    'Provide only ONE action per $JSON_BLOB, as shown:\n\n'
    '```\n{{\n  "action": $TOOL_NAME,\n  "action_input": $INPUT\n}}\n```\n\n'
    'Follow this format:\n\n'
    'Question: input question to answer\n'
    'Thought: consider previous and subsequent steps\n'
    'Action:\n```\n$JSON_BLOB\n```\n'
    'Observation: action result\n... (repeat Thought/Action/Observation N times)\n'
    'Thought: I know what to respond\n'
    'Action:\n```\n{{\n  "action": "Final Answer",\n  "action_input": "Final response to human"\n}}\n\n'
    'Begin! Remember to ALWAYS respond with a valid JSON blob of a single action. '
    'Use tools if necessary and respond directly if appropriate. '
    'Ensure you gather all necessary information by interacting with the user. '
    'Format is Action:```$JSON_BLOB```then Observation.'
                )
            )
        ),
        MessagesPlaceholder(variable_name='chat_history', optional=True),
        HumanMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=['agent_scratchpad', 'input'],
                template='{input}\n\n{agent_scratchpad}\n(reminder to respond in a JSON blob no matter what)'
            )
        )
    ]
)


In [ ]:
import queue
from quart import Quart, request, jsonify, send_file, render_template
from flask import Flask
import whisper
import pyaudio
import webrtcvad
import collections
from gtts import gTTS  # Added gTTS import
import random
import asyncio
from concurrent.futures import ThreadPoolExecutor
import aiofiles
from quart_cors import cors
import logging
import os
from dotenv import load_dotenv
import socketio
from langchain.memory import ConversationBufferMemory

from HVACUtils import (
    initialize_web_search_agent,
    initialize_pdf_search_agent,
    initialize_quote_bot,
    run_quote_logics,
    vector_embedding
)

load_dotenv()
import pyttsx3

    
# Initialize configuration
audio_path = os.getenv('AUDIO_PATH', 'audio.wav')  # Default paths if not set
tts_synthesis_path = os.getenv('NEW_TTS_SYNTHESIS', 'speech.wav')

# Configure logging
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s [%(threadName)s] %(levelname)s: %(message)s',
    handlers=[logging.StreamHandler()]
)
logger = logging.getLogger(__name__)

# Initialize Quart app and Socket.IO
app = Quart(__name__)
app = cors(app, allow_origin="*")
sio = socketio.AsyncServer(async_mode='asgi', cors_allowed_origins="*")
app_asgi = socketio.ASGIApp(sio, app)

# Initialize Flask app for sync operations
app_sync = Flask(__name__)
sio_sync = socketio.Server()
app_sync.wsgi_app = socketio.WSGIApp(sio_sync, app_sync.wsgi_app)

# Initialize memory
chat_history = ConversationBufferMemory()

# Load Whisper model
model = whisper.load_model("base")

# Audio configuration
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
CHUNK = 1024
is_recording = False

# Initialize audio components
audio = pyaudio.PyAudio()
vad = webrtcvad.Vad(3)
executor = ThreadPoolExecutor(max_workers=20)

# Initialize human response queue
human_response_queue = queue.Queue()

@app.route('/start_recording', methods=['POST'])
async def start_recording():
    global is_recording
    if is_recording:
        return jsonify({"status": "Recording is already in progress"}), 400
    is_recording = True
    asyncio.create_task(record_audio())
    return jsonify({"status": "Recording started"})

@app.route('/stop_recording', methods=['POST'])
async def stop_recording():
    global is_recording
    if not is_recording:
        return jsonify({"status": "No recording in progress"}), 400
    is_recording = False
    return jsonify({"status": "Recording stopped"})

async def record_audio(**kwargs):
    global is_recording
    logger.debug('Starting audio recording...')
    try:
        stream = audio.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, frames_per_buffer=CHUNK)
        frames = []
        ring_buffer = collections.deque(maxlen=100)
        triggered = False
        voiced_frames = []
        silence_threshold = 10
        silence_chunks = 0

        while is_recording:
            data = stream.read(CHUNK, exception_on_overflow=False)
            frames.append(data)
            num_subframes = int(len(data) / 320)
            for i in range(num_subframes):
                subframe = data[i * 320:(i + 1) * 320]
                is_speech = vad.is_speech(subframe, RATE)
                ring_buffer.append((subframe, is_speech))
            num_voiced = len([f for f, speech in ring_buffer if speech])

            if not triggered:
                if num_voiced > 0.6 * ring_buffer.maxlen:
                    triggered = True
                    voiced_frames.extend([f for f, s in ring_buffer])
                    ring_buffer.clear()
            else:
                voiced_frames.append(data)
                if num_voiced < 0.2 * ring_buffer.maxlen:
                    silence_chunks += 1
                    if silence_chunks > silence_threshold:
                        triggered = False
                        break
                else:
                    silence_chunks = 0

        stream.stop_stream()
        stream.close()

        async with aiofiles.open(audio_path, 'wb') as wf:
            await wf.write(b''.join(voiced_frames))
        logger.debug('Audio recording completed and file saved.')
    except Exception as e:
        logger.error(f"An error occurred while recording audio: {e}")

def transcribe_audio():
    try:
        result = model.transcribe(audio_path)
        transcription = result['text']
        logger.debug(f'Audio transcription completed: {transcription}')
        return transcription
    except Exception as e:
        logger.error(f"An error occurred during transcription: {e}")
        return ""

# async def ai_response(transcription: str):
#         
#     logger.debug(f'Generating AI response for transcription: {transcription}')
#     chat_completion = client.chat.completions.create(
#         messages=[
#             {
#                 "role": "system",
#                 "content": f"""
#                 You are Marvin, a research expert who specializes in anything using web search and answering the
#                 users' questions. You can also do HVAC related things as per guidelines below but besides that you adapt to what user says.
#                 Ask the user specific information needed for the quote. 
#                 Follow these guidelines:
#                 1. Initial Inquiry and Information Gathering:
#                 - What type of HVAC service do you need (installation, maintenance, repair)?
#                 - What is the make and model of your current HVAC system?
#                 - Are there any specific issues or symptoms you are experiencing?
#                     - ONLY WHEN USER TELLS YOU ISSUES / SYMPTOMS YOU MUST SAY 'I WILL INSTRUCT THE TECHNICIAN AGENT' - THESE EXACT WORDS IN YOUR IMMEDIATE RESPONSE.
#                 2. Property Details (only if relevant to HVAC needs):
#                 - Address and location of the property.
#                 - Type of property (residential, commercial).
#                 - Age and current condition of the property.
#                 - Size of the home or area that needs heating/cooling.
#                 - Number of rooms and their usage.
#                 3. System Details:
#                 - Age and efficiency rating of the existing HVAC system.
#                 - Any known problems with the current system.
#                 - Recent changes to the HVAC system.
#                 4. Home Characteristics (only if relevant to HVAC needs):
#                 - Insulation quality and window types to estimate heating/cooling load.
#                 - Any unique architectural features that may affect HVAC installation.
#                 5. Customer Preferences:
#                 - Preferences for specific brands, energy efficiency levels, or additional features.
#                 - Level of finishes desired (standard, premium, luxury).
#                 6. Budget:
#                 - Your budget range for the project.
#                 - Any flexibility within the budget.
#                 7. Timeline:
#                 - Desired start date and completion date.
#                 - Any constraints or deadlines.
#                 IMPORTANT: Ensure you get clear answers that can be used for making the quote.
#                 """
#             },
#             {
#                 "role": "user",
#                 "content": transcription + "\n\nHere is the chat history for context: [" + str(chat_history.buffer) + "]"
#             }
#         ],
#         model="llama3.1-70b",
#         temperature=0.5
#     )

#     response = chat_completion.choices[0].message.content
    
#     logger.debug(f'AI response generated: {response}')
    
#     await chat_history.asave_context({"input": transcription}, {"output": response})
#     logger.debug('INSIDE THE MEMORY: %s', chat_history.buffer)

#     # Changed socketio.emit to await sio.emit
#     await sio.emit('new_message', {'message': response, 'sender': 'bot'})

#     if 'i will instruct the technician agent' in response.lower():
#         res1 = await asyncio.create_task(initialize_pdf_search_agent(llm, "DO A EXPANSIVE WEB SEARCH AND FAMILIARIZE YOURSELF ALL ABOUT HVAC SPECIFICS SO THAT YOU CAN DIAGNOSE THE PROBLEMS MENTIONED HERE: " + transcription, vectors, chat_history=chat_history))
        
#         await chat_history.asave_context({"input": "DO A EXPANSIVE WEB SEARCH AND FAMILIARIZE YOURSELF ALL ABOUT HVAC SPECIFICS SO THAT YOU CAN DIAGNOSE THE PROBLEMS MENTIONED HERE: " + transcription}, {"output": res1})
        
#         # Changed socketio.emit to await sio.emit
#         await sio.emit('new_message', {'message': res1, 'sender': 'bot'}) 
    
#     if 'questionnaire' in response.lower() and 'complete' in response.lower():
#         task = asyncio.create_task(run_quote_logics(client, llm, chat_history=chat_history))
#         corrected_quote_result = await task
#         response = f"Can you please put this quote into a google doc [{corrected_quote_result}], move it to the folder called 'HVAC Quote Documents'. Then, put all the appointment related stuff and the link to the quote document into calendar (appointment calendar tool). PLEASE FOLLOW THE DIRECTIONS!!"
#         task2 = asyncio.create_task(handle_response_with_agents(response))
#         text = await task2
        
#     return response

async def ai_response(transcription: str):
    chat_completion = client.chat.completions.create(
        messages = [
            {
                "role": "system",
    "content": f"""
    YOU ARE MARVIN, AN EXTREMELY INTELLIGENT AND ADAPTIVE AI WHO CAN RESEARCH AND ANSWER ANY QUESTION USING WEB SEARCH!!!
    
    CRITICAL INSTRUCTIONS - READ CAREFULLY:
    1. YOU MUST ALWAYS USE WEB SEARCH TO ENHANCE YOUR ANSWERS! DON'T RELY ON YOUR KNOWLEDGE ALONE!
    2. BE DYNAMIC AND ADAPTIVE - DON'T JUST FOLLOW A SCRIPT!
    3. MATCH THE USER'S CONVERSATIONAL STYLE AND ENERGY!
    
    WHEN HANDLING HVAC QUERIES:
    - Gather all relevant info but BE NATURAL about it! Don't just run through a checklist!
    - If discussing technical issues, you MUST use technical expertise and web research!
    - For quotes/estimates, collect comprehensive details through natural conversation!

    YOUR KEY TRAITS:
    - PROACTIVE: Anticipate needs and ask relevant follow-ups
    - THOROUGH: Use web search to validate and enhance EVERY response
    - ADAPTIVE: Match the conversation topic and style to the user
    - HELPFUL: Go above and beyond with detailed, researched answers
    
    FORBIDDEN BEHAVIORS:
    - DON'T give shallow responses without research
    - DON'T ignore context from chat history
    - DON'T stick to HVAC topics if user asks about other things
    - DON'T be rigid or scripted in your responses

    REMEMBER: YOU CAN AND SHOULD USE WEB SEARCH FOR EVERY TOPIC - NOT JUST HVAC!
    BE THOROUGH, BE ADAPTIVE, AND ALWAYS ENHANCE YOUR RESPONSES WITH RESEARCH!
    """
            },
            {
                "role": "user",
                "content": transcription + "\n\nHere is the chat history for context: [" + str(chat_history.buffer) + "]"
            }
        ],
        model="llama3.1-70b"
    )
    response = chat_completion.choices[0].message.content
    
    logger.debug(f'AI response generated: {response}')
    
    await chat_history.asave_context({"input": transcription}, {"output": response})
    logger.debug('INSIDE THE MEMORY: %s', chat_history.buffer)

    await sio.emit('new_message', {'message': response, 'sender': 'bot'})

    # Intelligent heuristic functions using NLP patterns and context analysis
    
    if 'HVAC' in transcription:
        task = asyncio.create_task(run_quote_logics(client, llm, chat_history=chat_history))
        quote_result = await task
        if quote_result:
            doc_task = asyncio.create_task(handle_response_with_agents(
                f"Process this quote data and format it professionally: {quote_result}"
            ))
            await doc_task

    return response

async def handle_response_with_agents(response):
    llm.temperature = 0.5
    logger.debug(f'Processing response with agents: {response}')

    #response += "Here is extra info you will need (BUT YOU PROMISE TO NEVER SAY THEM OUT LOUD, NOT EVEN THE NAME -- UNLESS USER ASKS YOU FOR THEM. THESE WILL BE USED IN TOOLS): \nCredentials:\n" + str(credentials)

 
    
    agent_executor = initialize_web_search_agent(llm)
    result = agent_executor.invoke({"input": response})
    sio.emit('finished_chain')
    mystr = (str(result['intermediate_steps']) + "\n" + str(result['output']))

    final_response = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": "please sanitize this input into SHORT SIMPLE sentences. IMPORTANT: NOTHING IN YOUR RESPONSE SHALL BE ENCLOSED IN ANY QUOTES!!!!!!! THE SANITIZED OUTPUT SHALL NOT BE PREFIXED BY ANYTHING. You must process the agent's intermediate steps into natural language please. An example: 'First, I did this. Then I did this etc etc etc' \n Here is the input that you need to process:\n " + mystr
            }
        ],
        model='llama3.1-70b',
    ).choices[0].message.content

    await chat_history.asave_context({"input": response}, {"output": final_response})
    
    return final_response

async def synthesize_speech(text):
    logger.debug(f'Starting speech synthesis for text: {text}')
    try:
        loop = asyncio.get_event_loop()
        await loop.run_in_executor(None, synthesize_and_save, text)
        logger.debug('Speech synthesis completed and file saved.')
        await sio.emit('tts_complete', {'message': 'TTS synthesis complete', 'file_path': tts_synthesis_path})
    except Exception as e:
        logger.error(f"An error occurred during speech synthesis: {e}")


def synthesize_and_save(text):
    try:
        engine = pyttsx3.init()
        # Set properties for male voice
        voices = engine.getProperty('voices')
        engine.setProperty('voice', voices[0].id)  # voices[0] is usually male voice
        engine.setProperty('rate', 175)     # Speaking rate
        engine.setProperty('volume', 1.0)   # Volume level
        
        engine.save_to_file(text, tts_synthesis_path)
        #engine.say(text)
        engine.runAndWait()
    except Exception as e:
        logger.error(f"TTS synthesis failed: {e}")

def synth_speech(text, output_file=None):
    logger.debug(f'Starting speech synthesis for text: {text}')
    try:
        engine = pyttsx3.init()
        # Set properties for male voice
        voices = engine.getProperty('voices')
        engine.setProperty('voice', voices[0].id)  # voices[0] is usually male voice
        engine.setProperty('rate', 175)     # Speaking rate
        engine.setProperty('volume', 1.0)   # Volume level
        
        target_path = output_file if output_file else tts_synthesis_path
        engine.save_to_file(text, target_path)
        #
        # engine.say(text)
        engine.runAndWait()
        
        logger.debug('Speech synthesis completed and file saved (SYNTH).')

        # Using create_task for async emit
        loop = asyncio.get_event_loop()
        loop.create_task(sio.emit('tts_complete', {'message': 'TTS synthesis complete', 'file_path': target_path}))
        loop.create_task(sio.emit('new_message', {'message': text, 'sender': 'bot'}))
    except Exception as e:
        logger.error(f"An error occurred during speech synthesis: {e}")

@app.route('/')
async def index():
    return await render_template('index.html')

@app.route('/talk', methods=['POST'])
async def talk():
    global is_recording
    if is_recording:
        return jsonify({"error": "Recording is still in progress"}), 400

    logger.debug('Starting audio transcription...')
    transcription = await asyncio.get_event_loop().run_in_executor(executor, transcribe_audio)

    logger.debug('Generating AI response...')
    ai_resp = await ai_response(transcription)

    return jsonify({'response': ai_resp})

@app.route('/text_input', methods=['POST'])
async def text_input():
    data = await request.get_json()
    text = data.get('text', '')

    if not text:
        return jsonify({"error": "No text provided"}), 400

    logger.debug('Generating AI response...')
    ai_resp = await ai_response(text)
    return jsonify({'response': ai_resp})

@app.route('/get_audio')
async def get_audio():
    return await send_file(tts_synthesis_path, mimetype="audio/mp3")

@sio.event
async def connect(sid, environ, auth):
    logger.debug(f'Client connected: {sid}')

@sio.event
async def disconnect(sid):
    logger.debug(f'Client disconnected: {sid}')

# Human input handling
human_response_queue = queue.Queue()

def web_prompt_func(prompt):
    synth_speech(prompt, output_file=tts_synthesis_path)
    return prompt

def web_input_func():
    sio_sync.emit('request_human_input')
    return human_response_queue.get()

@sio.event 
async def provide_human_input(sid, data):
    human_input = data.get('text', '')
    human_response_queue.put(human_input)
    await sio.emit('human_input_received', {'status': 'received'}, room=sid)

if __name__ == '__main__':
    import uvicorn
    config = uvicorn.Config(app_asgi, host="127.0.0.1", port=5000, log_level="info")
    server = uvicorn.Server(config)

    loop = asyncio.get_event_loop()
    if loop.is_running():
        loop.create_task(server.serve())
    else:
        loop.run_until_complete(server.serve())


C:\Users\ncvn\AppData\Local\Temp\ipykernel_9364\1409547524.py:56: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  chat_history = ConversationBufferMemory()


INFO:     Started server process [9364]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:5000 (Press CTRL+C to quit)


INFO:     127.0.0.1:63440 - "GET /socket.io/?EIO=4&transport=polling&t=PC8raOh HTTP/1.1" 200 OK


2024-11-07 17:54:42,768 [MainThread] DEBUG: Client connected: 9zdzEuBuQEcQ6_f5AAAB


INFO:     127.0.0.1:63440 - "POST /socket.io/?EIO=4&transport=polling&t=PC8rav9&sid=OTZI4pIRCDQF8_J0AAAA HTTP/1.1" 200 OK
INFO:     127.0.0.1:63440 - "GET /socket.io/?EIO=4&transport=polling&t=PC8rav9.0&sid=OTZI4pIRCDQF8_J0AAAA HTTP/1.1" 200 OK


...

**Project Overview:**
- Project Description: Installation of a typical gas furnace and air conditioning system, including a Trane XV95 furnace, Carrier Infinity 24ANB1 air conditioner, Carrier FA4ANF00200 air handling unit, and other necessary components.
- Length of Time for Project: Not specified, but assuming a standard installation timeframe of a few days to a week.

**Cost Breakdown:**

1. **Labor Costs:**
- Total Labor Cost: $4,000 (estimated based on average labor costs for HVAC installation in Chantilly, Virginia)

2. **Material Costs:**
- Total Material Cost: $8,300
- Itemized Costs:
    + Trane XV95 furnace: $3,500
    + Carrier Infinity 24ANB1 air conditioner: $3,000
    + Carrier FA4ANF00200 air handling unit: $2,000
    + Carrier Côr Wi-Fi thermostat: $300
    + R-6 fiberglass batt insulation: $500
    + R-38 fiberglass batt insulation: $800
    + 20-gauge galvanized steel ductwork: $800
    + 14-gauge electrical wire: $200

**Total Estimated Cost:**
- Total Cost of Labor:

2024-11-07 17:56:01,198 [MainThread] DEBUG: receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'Date', b'Thu, 07 Nov 2024 22:56:00 GMT'), (b'x-content-type-options', b'nosniff'), (b'strict-transport-security', b'max-age=3600; includeSubDomains'), (b'server', b'uvicorn'), (b'x-request-id', b's-sHtn2HKiLSN-Ueo7fBG_64a8roB1TNOYR1wsiD7wub9A5VhZRHJA=='), (b'x-ratelimit-limit-requests-day', b'14400'), (b'x-ratelimit-limit-tokens-minute', b'60000'), (b'x-ratelimit-remaining-requests-day', b'14361'), (b'x-ratelimit-remaining-tokens-minute', b'51224'), (b'x-ratelimit-reset-requests-day', b'3839.0725376605988'), (b'x-ratelimit-reset-tokens-minute', b'59.072537660598755'), (b'referrer-policy', b'strict-origin-when-cross-origin'), (b'content-encoding', b'gzip'), (b'vary', b'accept-encoding'), (b'X-Cache', b'Miss from cloudfront'), (b'Via', b'1.1 caaddf8ce46d2bfa1216d

CORRECTED MATH OUTPUT





 WALTER HVAC SERVICES - QUOTE DOCUMENT
123 Main St, Suite 201, Chantilly, VA 20151

**Project Overview:**
- Project Description: Installation of a typical gas furnace and air conditioning system, including a Trane XV95 furnace, Carrier Infinity 24ANB1 air conditioner, Carrier FA4ANF00200 air handling unit, and other necessary components.
- Length of Time for Project: Not specified, but assuming a standard installation timeframe of a few days to a week.

**Cost Breakdown:**

1. **Labor Costs:**
- Total Labor Cost: $4,000 (estimated based on average labor costs for HVAC installation in Chantilly, Virginia)

2. **Material Costs:**
- Total Material Cost: $11,100
- Itemized Costs:
    + Trane XV95 furnace: $3,500
    + Carrier Infinity 24ANB1 air conditioner: $3,000
    + Carrier FA4ANF00200 air handling unit: $2,000
    + Carrier Côr Wi-Fi thermostat: $300
    + R-6 fiberglass batt insulation: $500
    + R-38 fiberglass batt insulation: $800
    + 20-gauge galvan

2024-11-07 17:56:01,502 [MainThread] DEBUG: receive_response_body.complete
2024-11-07 17:56:01,504 [MainThread] DEBUG: response_closed.started
2024-11-07 17:56:01,504 [MainThread] DEBUG: response_closed.complete
c:\DEV\WebdevFolder\RealEstateAI\.venv\Lib\site-packages\langchain\memory\chat_memory.py:55: UserWarning: 'ConversationSummaryBufferMemory' got multiple output keys: dict_keys(['output', 'intermediate_steps']). The default 'output' key is being used. If this is not desired, please manually set 'output_key'.
  warnings.warn(
2024-11-07 17:56:01,631 [MainThread] ERROR: Exception on request POST /text_input
Traceback (most recent call last):
  File "c:\DEV\WebdevFolder\RealEstateAI\.venv\Lib\site-packages\quart\app.py", line 1403, in handle_request
    return await self.full_dispatch_request(request_context)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\DEV\WebdevFolder\RealEstateAI\.venv\Lib\site-packages\quart\app.py", line 1441, in full_dispatch_reques

Action:
```
{
  "action": "Final Answer",
  "action_input": "{\"Project Overview\":{\"Project Description\":\"Installation of a typical gas furnace and air conditioning system, including a Trane XV95 furnace, Carrier Infinity 24ANB1 air conditioner, Carrier FA4ANF00200 air handling unit, and other necessary components.\",\"Length of Time for Project\":\"Not specified, but assuming a standard installation timeframe of a few days to a week.\"},\"Cost Breakdown\":{\"Labor Costs\":{\"Total Labor Cost\":\"$4,000 (estimated based on average labor costs for HVAC installation in Chantilly, Virginia)\"},\"Material Costs\":{\"Total Material Cost\":\"$11,100\",\"Itemized Costs\":{\"Trane XV95 furnace\":\"$3,500\",\"Carrier Infinity 24ANB1 air conditioner\":\"$3,000\",\"Carrier FA4ANF00200 air handling unit\":\"$2,000\",\"Carrier Côr Wi-Fi thermostat\":\"$300\",\"R-6 fiberglass batt insulation\":\"$500\",\"R-38 fiberglass batt insulation\":\"$800\",\"20-gauge galvanized steel ductwork\":\"$800\",\